In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
pip install torch torchvision wandb


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


# **Question 1 & Question 2**

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler, random_split
from torchvision.datasets import ImageFolder
import wandb
from wandb.sdk.wandb_run import Run
from tqdm import tqdm

# Load the dataset
train_data = ImageFolder('/kaggle/input/dataset1/inaturalist_12K/train', transform=train_transforms)
test_data = ImageFolder('/kaggle/input/dataset1/inaturalist_12K/val', transform=val_transforms)

In [6]:
# Set up data transformations
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Count the number of samples in each class
class_counts = {}
for _, label in train_data:
    if label not in class_counts:
        class_counts[label] = 0
    class_counts[label] += 1

# Calculate the number of samples per class for validation set
val_size_per_class = {label: int(count * 0.2) for label, count in class_counts.items()}

# Initialize lists to hold indices for train and validation sets
train_indices = []
val_indices = []

# Iterate through the dataset and assign samples to train or validation set
for idx, (_, label) in enumerate(train_data):
    if val_size_per_class[label] > 0:
        val_indices.append(idx)
        val_size_per_class[label] -= 1
    else:
        train_indices.append(idx)

# Create SubsetRandomSampler for train and validation sets
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
    

1
2
3
4


In [ ]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self, filters, activation, filter_organization, data_augmentation, batch_norm, dropout):
        super(CNN, self).__init__()

        self.conv_layers = nn.Sequential()
        self.dense_layers = nn.Sequential()

        # Add conv layers based on filter organization
        if filter_organization == 'same':
            num_filters = [filters] * 5  # Assuming 5 convolution layers
        elif filter_organization == 'double':
            num_filters = [filters * 2**i for i in range(5)]  # Doubling filters in each subsequent layer
        else:
            num_filters = [filters // 2**i for i in range(5)]  # Halving filters in each subsequent layer

        in_channels = 3
        for i, f in enumerate(num_filters):
            print(f)
            self.conv_layers.add_module(f"conv_{i}", nn.Conv2d(in_channels, f, kernel_size=3, padding=1))
            if batch_norm:
                self.conv_layers.add_module(f"batch_norm_{i}", nn.BatchNorm2d(f))
            self.conv_layers.add_module(f"{activation}_{i}", getattr(nn, activation)())
            self.conv_layers.add_module(f"maxpool_{i}", nn.MaxPool2d(kernel_size=2))
            if data_augmentation and i == 0:
                self.conv_layers.add_module("dropout", nn.Dropout2d(p=dropout))  # Dropout only in the first layer
            in_channels = f

        self.dense_layers.add_module("flatten", nn.Flatten())
        self.dense_layers.add_module("dense", nn.Linear(in_channels * 7 * 7, 512))
        self.dense_layers.add_module("relu", nn.ReLU())
        self.output_layer = nn.Linear(512, len(train_data.classes))

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.dense_layers(x)
        x = self.output_layer(x)
        return x

In [ ]:
# Function to calculate accuracy
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    accuracy = correct / labels.size(0)
    return accuracy

In [ ]:
# Training loop
def training_model(optimizer, criterion, model, train_loader, val_loader):
    epochs = 5
    for epoch in range(epochs):
        model.train()
        training_loss = 0.0
        train_accuracy = 0.0
        pbar = tqdm(train_loader, total=len(train_loader))
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
            train_accuracy += calculate_accuracy(outputs, labels)


        model.eval()
        val_loss = 0.0
        val_accuracy = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()
                val_accuracy += calculate_accuracy(outputs, labels)

        train_accuracy /= len(train_loader)
        training_loss /= len(train_loader)
        val_loss /= len(val_loader)
        val_accuracy /= len(val_loader)
        pbar.set_description(f'Epoch {epoch+1}/{epochs}, Train_Loss: {training_loss:.4f},  Train_Acc: {train_accuracy:.4f},  Val_Loss: {val_loss:.4f},  Val_Loss: {val_accuracy:.4f}')

        wandb.log({"epoch": epoch+1, "train_loss": training_loss, "val_loss": val_loss, "val_accuracy": val_accuracy, "train_accuracy": train_accuracy})

# Test the model
# model.eval()
# test_accuracy = 0.0
# with torch.no_grad():
#     for images, labels in test_loader:
#         outputs = model(images)
#         test_accuracy += calculate_accuracy(outputs, labels)

# test_accuracy /= len(test_loader)
# wandb.log({"test_accuracy": test_accuracy})

# Save the model
# torch.save(model.state_dict(), 'model.pth')

In [ ]:
sweep_config = {
    'method': 'bayes',  # Random search method
    'metric': {'goal': 'maximize', 'name': 'val_accuracy'},  # Metric to optimize
    'parameters': {
        'batch_size': {32, 64},
        'num_filters': {'values': [32, 64, 128]},
        'activation': {'values': ['ReLU', 'GELU', 'SiLU', 'Mish']},
        'filter_organization': {'values': ['same', 'double', 'halve']},
        'data_augmentation': {'values': [True, False]},
        'batch_norm': {'values': [True, False]},
        'dropout': {'values': [0.2, 0.3]}
    }
}

In [ ]:
import wandb

sweep_id = wandb.sweep(sweep_config, project='your-project-name', entity='your-entity-name')

# Define your training function
def train():
    # Use wandb.config to access hyperparameters in your training script
    config = wandb.config
    num_filters = config.num_filters
    activation = config.activation
    filter_organization = config.filter_organization
    data_augmentation = config.data_augmentation
    batch_norm = config.batch_norm
    batch_size = config.batch_size
    dropout = config.dropout
    
    # Your training code here
    # Create an instance of the CNN model
    model = CNN(num_filters, activation, filter_organization, data_augmentation, batch_norm, dropout)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Create DataLoader instances for train and validation sets using the samplers
    train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(train_data, batch_size=batch_size, sampler=val_sampler)
    
    training_model(optimizer, criterion, model, train_loader, val_loader)

    
# Initialize Wandb run
wandb.init()

# Run the sweep
wandb.agent(sweep_id, function=train)
